In [ ]:
%matplotlib notebook
import numpy as np
import math
from scipy.misc import factorial
import matplotlib.pyplot as plt
from ipywidgets import widgets
from IPython.display import display
import scipy.stats

### One-sample t-test

In this example we show an interactive demo of how to perform a hypothesis test of the population mean based on the t-distribution. We use the t-distribution to test the population mean when the sampling distribution of the point estimate of interest (in this case the sample mean) is nearly normally distributed and the sample is small.

The thicker tails of the t-distribution, compared to those of the normal distribution, is the correction factor we need to compensate for the small size of the sample. 

We follow the same process followed for the case of hypothesis testing based on the normal model, but using the t-distribution instead. The T-score is the equivalent of the Z-score in this context, and it is calculated exactly in the same way. 

The one-sample t-test is commonly applied to large samples as well because the results are then very similar to those of hypothesis testing based on the normal model. This is because the t-distribution becomes practically indistinguishible from the normal distribution when the value of the degrees of freedom is higher than 30. 

In [ ]:
fig_one, ax_one = plt.subplots()
ax_one.grid(True)
plt.ion()

POINTS_ONE = 1000

def my_label(text, width='20%'):
    return widgets.Label(text, layout=widgets.Layout(width=width))  

pm_one = widgets.FloatText(value=0)
sm_one = widgets.FloatText(value=0.1)
ssd_one = widgets.FloatText(value=1)
ss_one = widgets.FloatText(value=10, min=3)
sl_one = widgets.FloatText(value=0.05, min=0.01, max=0.99)
osts_one = widgets.RadioButtons(options=['One-sided (left)', 'One-sided (right)', 'Two-sided'])
result1_one = my_label('', width='100%') 
result2_one = my_label('', width='100%') 

box_one = widgets.VBox([
    my_label('Population'),
    widgets.HBox([my_label('Mean (null hypothesis)'), pm_one]),
    my_label('Sample'),
    widgets.HBox([my_label('Mean'), sm_one]),
    widgets.HBox([my_label('Standard deviation'), ssd_one]),
    widgets.HBox([my_label('Size'), ss_one]),
    my_label('Hypothesis test'), 
    widgets.HBox([my_label('Significance level'), sl_one]),
    osts_one,
    result1_one,
    result2_one
])

# Plots the t-distribution distribution centered around the null hypothesis and with
# standard deviation = standard error of the sampling distribution. The area of the curve 
# corresponding to the alternate hypothesis is highlighted
def on_change_one(change):
    global fig_one, ax_one
    
    # Auxiliar functions
    def compute_pdf(x, mean, std, df):
        def gamma(n):
            return factorial(n-1)
        return gamma((df+1)/2)/(math.sqrt(math.pi*df)*std_one*gamma(df/2))*(1 + ((x-mean_one)/std_one)**2/df)**-((df+1)/2)
    
    def plot_alternate_hypothesis(start, end, mean, std, df, ax):
        if start == 0 and end == 0:
            x = np.array([0])
        else:
            x = np.arange(start, end, (end - start)/POINTS_ONE)
        y = compute_pdf(x, mean, std, df)
        ax.fill_between(x, 0, y, color='red', alpha=0.3)
    
    # Deleting previous figure
    while len(ax_one.lines) > 0:
        l = ax_one.lines.pop(0)
        del l
    while len(ax_one.collections) > 0:
        c = ax_one.collections.pop(0)
        del c
        
    # Plotting - The null hypothesis is distributed following a t-distribution which mean is the 
    # assumed population mean, standard deviation is the standard error of the sample, and
    # degrees of freedom is equal to the number of observations in the sample minus one
    mean_one = pm_one.value
    std_one = ssd_one.value / math.sqrt(ss_one.value)
    df_one = ss_one.value - 1
    x_one = np.arange(mean_one-3*std_one, mean_one+3*std_one, 6*std_one/POINTS_ONE)
    y_one = compute_pdf(x_one, mean_one, std_one, df_one)
    ax_one.plot(x_one,y_one,color='blue')
    ax_one.set_xlim([mean_one-3*std_one, mean_one+3*std_one])
    
    # Plotting - red vertical line for the alternate hypothesis
    if osts_one.value == 'One-sided (left)':
        plot_alternate_hypothesis(-3*std_one, sm_one.value, mean_one, std_one, df_one, ax_one)
        p_value_one = scipy.stats.t(loc=mean_one, scale=std_one, df=df_one).cdf(sm_one.value)
    elif osts_one.value == 'One-sided (right)':
        plot_alternate_hypothesis(sm_one.value, 3*std_one, mean_one, std_one, df_one, ax_one)
        p_value_one = 1 - scipy.stats.t(loc=mean_one, scale=std_one, df=df_one).cdf(sm_one.value)
    else:
        if sm_one.value >= pm_one.value:
            plot_alternate_hypothesis(-3*std_one, -sm_one.value, mean_one, std_one, df_one, ax_one)
            plot_alternate_hypothesis(sm_one.value, 3*std_one, mean_one, std_one, df_one, ax_one)
            p_value_one = 2*scipy.stats.t(loc=mean_one, scale=std_one, df=df_one).cdf(-sm_one.value)
        elif sm_one.value < pm_one.value:
            plot_alternate_hypothesis(-3*std_one, sm_one.value, mean_one, std_one, df_one, ax_one)
            plot_alternate_hypothesis(-sm_one.value, 3*std_one, mean_one, std_one, df_one, ax_one)
            p_value_one = 2*scipy.stats.t(loc=mean_one, scale=std_one, df=df_one).cdf(sm_one.value)
            
    # Result of the hypothesis test
    result1_one.value = 'Result of the test: p-value = ' + str(p_value_one)
    if p_value_one < sl_one.value:
        result2_one.value = 'We reject the null hypothesis'
    else:
        result2_one.value = 'We fail to reject the null hypothesis'
        
    fig_one.canvas.draw()
    
wid_one = [pm_one, sm_one, ssd_one, ss_one, sl_one, osts_one]
[w.observe(on_change_one) for w in wid_one]
on_change_one(None)

display(box_one)

### One-sample t-test (paired data)

We say that we have paired data when we work with two datasets in which one observation from one dataset corresponds to exactly another single observation from the second dataset. To test whether there is a difference between the mean of the two populations we create a derived dataset by substracting each observation from the first sample to the corresponding observation from the second sample, and we perform a one-sample t-test in which the null hypothesis is that the mean of the derived dataset is equal to zero.

### Two-sample t-test

The two-sample t-test is aimed at determining whether there is a difference between the mean of two populations when data is not paired. The point estimate is calculated from a sample from each population by substracting their sample means. It is aimed at extracting conclusions from experiments: "is there a difference between the treatment and the control groups?". In this setting, the null hypothesis considers that the value of the difference of means is zero. 

In this example we will assume that we have some evidence that the standard deviation of the two populations is approximately the same. That's the reason why the demo only lets you set the value of the standard deviation of one of the samples and then creates a random very similar standard deviation value for the second sample. 

Given this assumption we can use the pooled variance method to improve the estimation of the standard deviation of the sampling distributions. The pooled variance method also lets us assign a higher value to the degrees of freedom parameter of the t-distribution, therefore letting us apply the test on a t-distribution that looks closer to a normal distribution. 